## Tic-Tac-Toe Agent
​
In this notebook, you will learn to build an RL agent (using Q-learning) that learns to play Numerical Tic-Tac-Toe with odd numbers. The environment is playing randomly with the agent, i.e. its strategy is to put an even number randomly in an empty cell. The following is the layout of the notebook:
        - Defining epsilon-greedy strategy
        - Tracking state-action pairs for convergence
        - Define hyperparameters for the Q-learning algorithm
        - Generating episode and applying Q-update equation
        - Checking convergence in Q-values

#### Importing libraries
Write the code to import Tic-Tac-Toe class from the environment file

In [283]:
# from <TC_Env> import <TicTacToe> - import your class from environment file
from TCGame_Env_v2 import TicTacToe
import collections
import numpy as np
import random
import pickle
import time
from matplotlib import pyplot as plt

In [284]:
# Function to convert state array into a string to store it as keys in the dictionary
# states in Q-dictionary will be of form: x-4-5-3-8-x-x-x-x
#   x | 4 | 5
#   ----------
#   3 | 8 | x
#   ----------
#   x | x | x

def Q_state(state):

    return ('-'.join(str(e) for e in state)).replace('nan','x')

In [285]:
# Defining a function which will return valid (all possible actions) actions corresponding to a state
# Important to avoid errors during deployment.

def valid_actions(curr_state):

    valid_Actions = []
    
    valid_Actions = [i for i in env.action_space(curr_state)[0]] ###### -------please call your environment as env
    return valid_Actions

In [286]:
# Defining a function which will add new Q-values to the Q-dictionary. 
def add_to_dict(state):
    state1 = Q_state(state)
    
    valid_act = valid_actions(state)
    if state1 not in Q_dict.keys():
        for action in valid_act:
            Q_dict[state1][action]=0

#### Epsilon-greedy strategy - Write your code here

(you can build your epsilon-decay function similar to the one given at the end of the notebook)

In [287]:
# Defining epsilon-greedy policy. You can choose any function epsilon-decay strategy
def epsilon_greedy(state, time):
    max_epsilon = 1.0
    min_epsilon = 0.001
    #time = np.arange(0,5000000)
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.000001*i)
    
    z = np.random.random()
    if z > epsilon:
        action = max(Q_dict[Q_state(state)])   #Exploitation: this gets the action corresponding to max q-value of current state
    else:
        action = random.choice(valid_actions(state))
    return action




#### Tracking the state-action pairs for checking convergence - write your code here

In [288]:
# Initialise Q_dictionary as 'Q_dict' and States_tracked as 'States_track' (for convergence)
Q_dict = collections.defaultdict(dict)

States_track = collections.defaultdict(dict)

rewards_tracked = {Q_state([1,3,0,0,0,0,0,0,0]):10,Q_state([1,0,0,0,3,0,0,0,0]):13}


print(len(Q_dict))
print(len(rewards_tracked))
print(len(States_track))

0
2
0


In [289]:
# Initialise states to be tracked
def initialise_tracking_states():
    sample_q_values = [(Q_state([1,3,0,0,0,0,0,0,0]),(5,7)),(Q_state([1,0,0,0,3,0,0,0,0]),(3,5))]    #select any 2 Q-values
    for q_values in sample_q_values:
        state = q_values[0]
        action = q_values[1]
        States_track[state][action] = {}
  


In [290]:
#Defining a function to save the Q-dictionary as a pickle file

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [291]:
def save_tracking_states():
    for state in States_track.keys():
        for action in States_track[state].keys():
            if state in Q_dict and action in Q_dict[state]:
                States_track[state][action].append(Q_dict[state][action])

In [292]:
initialise_tracking_states()

#### Define hyperparameters  ---write your code here

In [293]:
#EPISODES = 2000
EPISODES = 5000000
#STEPS = 2
STEPS = 3                 #for max of 3 steps only values
LR = 0.01                   #learning rate
GAMMA = 0.91


#threshold = 2000       #every these many episodes, the 4 Q-values will be stored/appended (convergence graphs)
threshold = 200
policy_threshold = 300    #every these many episodes, the Q-dict will be updated
#policy_threshold = 3

### Q-update loop ---write your code here

In [ ]:
start_time = time.time()

for episode in range(EPISODES):
    ##### Start writing your code from the next line
    
    
    #for episode in range(0,EPISODES):
#for episode in range(6393884,EPISODES):

    
    env = TicTacToe()      #creating an instance of the class
    #this initialises the state back to some initial value which is required for every new episode
    #this also initialises all parameters like m,p,K, etc.
    
    initial_state = env.state    #for the purpose of tracking reward
    curr_state = env.state     #curr_state is state_start
    #curr_state = [np.nan for _ in range(9)]
    add_to_dict(curr_state)
    
    #reward is not calculated for the immediately
    time_step = 0
    reward = None
    #curr_action = np.random.choice(np.arange(0,m+1))    #randomly choosing and action
    curr_action = random.choice(valid_actions(curr_state))
    
    next_state, reward, terminal = env.step(curr_state, curr_action)   #returned next state
    
    add_to_dict(next_state)

    curr_state = next_state           
    
    total_reward = 0
    
    
    while time_step < STEPS:    #the episode will run only for a few steps and not infinitely
        
        #curr_action is next choice and is a(t-1)      
        curr_action = epsilon_greedy(curr_state, episode)
        
        next_state, reward, terminal = env.step(curr_state, curr_action)
        #next_state is state if not terminal 
        
        add_to_dict(next_state)
            
        # UPDATE RULE
        max_next = max(Q_dict[Q_state(next_state)],key=Q_dict[Q_state(next_state)].get)   #this gets the action corresponding to max q-value of next state
        Q_dict[Q_state(curr_state)][max_next] += LR * ((reward + (GAMMA*(Q_dict[Q_state(next_state)][max_next])))) 


        curr_state = next_state       #state(t) became state(t-1)

        total_reward += reward
        
        time_step += 1

    
    #TRACKING REWARDS
    if Q_state(initial_state) in rewards_tracked:     #storing rewards
        rewards_tracked[Q_state(initial_state)] = total_reward
        save_obj(rewards_tracked,'Rewards')

    if ((episode+1) % threshold) == 0:   #every 2000th episode
        save_obj(rewards_tracked,'Rewards')   
    
    #TRACKING Q-VALUES
    if (episode == threshold-1):        #at the 1999th episode
        initialise_tracking_states()
      
    if ((episode+1) % threshold) == 0:   #every 2000th episode
        save_tracking_states()
        save_obj(States_track,'States_tracked')   
    
    #SAVING POLICY
    if ((episode+1)% policy_threshold ) == 0:  #every 30000th episodes, the Q-dict will be saved
        save_obj(Q_dict,'Policy')    
    
    
elapsed_time = time.time() - start_time
save_obj(rewards_tracked,'Rewards') 
save_obj(States_track,'States_tracked')   
save_obj(Q_dict,'Policy')

#### Check the Q-dictionary

In [ ]:
Q_dict

In [ ]:
len(Q_dict)

In [ ]:
# try checking for one of the states - that which action your agent thinks is the best  -----This will not be evaluated

#### Check the states tracked for Q-values convergence
(non-evaluative)

In [ ]:
# Write the code for plotting the graphs for state-action pairs tracked

### Epsilon - decay check

In [ ]:
max_epsilon = 1.0
min_epsilon = 0.001
time = np.arange(0,5000000)
epsilon = []
for i in range(0,5000000):
    epsilon.append(min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.000001*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()